# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'08-29-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'08-29-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-30 04:28:22,33.93911,67.709953,38143,1402,29063,7678.0,Afghanistan,97.982598,3.675642
1,NaN,NaN,NaN,Albania,2020-08-30 04:28:22,41.15330,20.168300,9279,275,5020,3984.0,Albania,322.433804,2.963681
2,NaN,NaN,NaN,Algeria,2020-08-30 04:28:22,28.03390,1.659600,43781,1491,30717,11573.0,Algeria,99.840271,3.405587
3,NaN,NaN,NaN,Andorra,2020-08-30 04:28:22,42.50630,1.521800,1124,53,902,169.0,Andorra,1454.733709,4.715302
4,NaN,NaN,NaN,Angola,2020-08-30 04:28:22,-11.20270,17.873900,2551,107,1041,1403.0,Angola,7.761757,4.194434


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37856,37894,37953,37999,38054,38070,38113,38129,38140,38143
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7967,8119,8275,8427,8605,8759,8927,9083,9195,9279
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,40258,40667,41068,41460,41858,42228,42619,43016,43403,43781


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1385,1385,1385,1387,1389,1397,1401,1401,1402,1402
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,238,240,245,250,254,259,263,266,271,275
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1411,1418,1424,1435,1446,1456,1465,1475,1483,1491


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,27681,28016,28016,28180,28360,28440,29042,29046,29059,29063
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3986,4096,4184,4332,4413,4530,4633,4791,4923,5020
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,28281,28587,28874,29142,29369,29587,29886,30157,30436,30717


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1240,1255,1264,1266,1286,1286,1281,1284,1296,1309
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,3957,3997,4037,4069,4083,4100,4166,4199,4230,4307
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,619,622,626,629,633,628,616,614,620,624


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,22,22,22,22,22,22,22,22,22,22
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,31,32,32,32,32,32,33,34,35,36
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
631,1001.0,Autauga,Alabama,US,2020-08-30 04:28:22,32.539527,-86.644082,1309,22,0,1287.0,"Autauga, Alabama, US",2342.980902,1.680672
668,1075.0,Lamar,Alabama,US,2020-08-30 04:28:22,33.779950,-88.096680,268,2,0,266.0,"Lamar, Alabama, US",1941.325607,0.746269
669,1077.0,Lauderdale,Alabama,US,2020-08-30 04:28:22,34.901719,-87.656247,1398,23,0,1375.0,"Lauderdale, Alabama, US",1507.618976,1.645207


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,5961094,182761,2140614
Brazil,3846153,120262,3202185
India,3542733,63498,2713933


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,5961094,182761,2140614,3637719,2020-08-30 04:28:22,37.919625,-91.366793
Brazil,3846153,120262,3202185,523706,2020-08-30 04:28:22,-12.669522,-48.480493
India,3542733,63498,2713933,765302,2020-08-30 04:28:22,23.088275,81.806127
Russia,982573,16977,802365,163231,2020-08-30 04:28:22,54.546312,62.120860
Peru,629961,28471,438017,163473,2020-08-30 04:28:22,-10.637324,-75.277744


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,702038,12894,0
Texas,626751,12607,0
Florida,619003,11105,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,702038,12894,0,689144,2020-08-30 04:28:22,37.843962,-120.728594
Texas,626751,12607,0,614144,2020-08-30 04:28:22,31.660643,-98.653069
Florida,619003,11105,0,607898,2020-08-30 04:28:22,28.940755,-82.700744
New York,433402,32938,0,400464,2020-08-30 04:28:22,42.671593,-75.579694
Georgia,267758,5576,0,262182,2020-08-30 04:28:22,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,239756,5759,0
New York,New York City,233688,23683,0
Florida,Miami-Dade,156038,2399,0
Arizona,Maricopa,133389,2950,0
Illinois,Cook,125124,5046,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,239756,5759,0,233997,2020-08-30 04:28:22,34.308284,-118.228241,6037.0
New York,New York City,233688,23683,0,210005,2020-08-30 04:28:22,40.767273,-73.971526,36061.0
Florida,Miami-Dade,156038,2399,0,153639,2020-08-30 04:28:22,25.611236,-80.551706,12086.0
Arizona,Maricopa,133389,2950,0,130439,2020-08-30 04:28:22,33.348359,-112.491815,4013.0
Illinois,Cook,125124,5046,0,120078,2020-08-30 04:28:22,41.841448,-87.816588,17031.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,38129,9083,43016,1098,2415,94,380292,43270,25448,26361,...,332509,1551,40447,42898,1036,20677,10,1933,11601,6292
2020-08-28,38140,9195,43403,1124,2471,94,392009,43451,25547,26590,...,333806,1556,40720,43879,1038,21251,10,1943,11779,6388
2020-08-29,38143,9279,43781,1124,2551,94,401239,43626,25670,26985,...,334916,1570,41067,44946,1040,21668,10,1946,11902,6406


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,1401,266,1475,53,105,3,8050,864,583,733,...,41564,43,298,358,30,141,1,562,282,189
2020-08-28,1402,271,1483,53,106,3,8271,869,600,733,...,41573,43,302,366,30,145,1,563,283,195
2020-08-29,1402,275,1491,53,107,3,8353,872,611,733,...,41585,44,307,375,32,147,1,563,284,196


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,29046,4791,30157,893,1335,89,274458,36988,20365,22317,...,1564,1333,37200,34147,637,14194,8,1098,10840,5010
2020-08-28,29059,4923,30436,902,1028,89,280165,37264,20631,22594,...,1615,1352,37873,34972,663,14291,8,1107,10945,5043
2020-08-29,29063,5020,30717,902,1041,89,287220,37563,20858,22866,...,1622,1364,38250,36175,677,14631,8,1113,11092,5056


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1240,1255,1264,1266,1286,1286,1281,1284,1296,1309
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,255,259,259,260,261,264,266,266,266,268
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1287,1308,1320,1329,1339,1353,1364,1373,1384,1398
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,383,389,396,402,411,419,427,432,433,442
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,2899,2908,2916,2930,2940,2946,3006,3031,3091,3200


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,121023,4974,0,200139,58745,691821,56323,52350,16976,49,...,149179,614549,50557,1586,1052,116569,72703,9633,73138,3733
2020-08-28,122185,5092,0,200658,59583,697385,56753,52495,17083,49,...,150815,619354,50948,1589,1075,117578,73301,9826,73981,3803
2020-08-29,123889,5182,0,201287,60378,702038,57020,52495,17349,49,...,152280,626751,51406,1605,1118,118794,73879,9970,74800,3795


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-27        1284    4199     614  515    979     527    813    2225   
2020-08-28        1296    4230     620  517    983     530    814    2255   
2020-08-29        1309    4307     624  523   1004     535    827    2316   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-27          860      336  ...     160      7      127       48   
2020-08-28          863      336  ...     160      7      141       48   
2020-08-29          867      341  ...     159      7      147       48   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-27            295   417   289         53      108     11  
2020-08-28            296   424   292         53      108     11  
2020-08-29            296   425   292         73      108     11  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,2076,37,0,4929,739,12677,1931,4465,604,0,...,1673,12365,403,58,14,2527,1890,198,1111,37
2020-08-28,2107,37,0,4978,756,12805,1937,4465,604,0,...,1701,12537,407,58,14,2550,1905,201,1113,37
2020-08-29,2152,37,0,5007,772,12894,1942,4465,604,0,...,1725,12607,407,58,14,2568,1905,215,1119,37


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-08-27          22      34       7    5      7      13     36      26   
2020-08-28          22      35       7    5      9      13     36      27   
2020-08-29          22      36       7    6      9      13     36      29   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-08-27           38       11  ...       0      0        0        0   
2020-08-28           39       11  ...       0      0        0        0   
2020-08-29           39       11  ...       0      0        0        0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-27              0     0     0         36        0      0  
2020-08-28              0     0     0         36        0      0  
2020-08-29              0     0     0         36        0      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.000420,0.017475,0.009315,0.000000,0.035592,0.0,0.027294,0.004714,0.004976,0.012599,...,0.004659,0.005185,0.012086,0.022233,0.001934,0.025899,0.0,0.001554,0.019778,0.006559
2020-08-28,0.000288,0.012331,0.008997,0.023679,0.023188,0.0,0.030811,0.004183,0.003890,0.008687,...,0.003901,0.003224,0.006750,0.022868,0.001931,0.027760,0.0,0.005173,0.015344,0.015257
2020-08-29,0.000079,0.009135,0.008709,0.000000,0.032376,0.0,0.023545,0.004028,0.004815,0.014855,...,0.003325,0.008997,0.008522,0.024317,0.001927,0.019623,0.0,0.001544,0.010442,0.002818


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.000000,0.011407,0.006826,0.0,0.019417,0.0,0.026917,0.003484,0.019231,0.0,...,0.000289,0.000000,0.024055,0.019943,0.000000,0.029197,0.0,0.003571,0.000000,0.055866
2020-08-28,0.000714,0.018797,0.005424,0.0,0.009524,0.0,0.027453,0.005787,0.029160,0.0,...,0.000217,0.000000,0.013423,0.022346,0.000000,0.028369,0.0,0.001779,0.003546,0.031746
2020-08-29,0.000000,0.014760,0.005394,0.0,0.009434,0.0,0.009914,0.003452,0.018333,0.0,...,0.000289,0.023256,0.016556,0.024590,0.066667,0.013793,0.0,0.000000,0.003534,0.005128


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.000138,0.034103,0.009068,0.000000,0.366428,0.0,0.021045,0.007134,0.013234,0.007767,...,0.008382,0.008321,0.021922,0.036926,0.007911,0.019025,0.0,0.000912,0.013747,0.001800
2020-08-28,0.000448,0.027552,0.009252,0.010078,-0.229963,0.0,0.020794,0.007462,0.013062,0.012412,...,0.032609,0.014254,0.018091,0.024160,0.040816,0.006834,0.0,0.008197,0.009686,0.006587
2020-08-29,0.000138,0.019703,0.009232,0.000000,0.012646,0.0,0.025182,0.008024,0.011003,0.012039,...,0.004334,0.008876,0.009954,0.034399,0.021116,0.023791,0.0,0.005420,0.013431,0.002578


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,0.014834,0.016139,NaN,0.003409,0.012443,0.007012,0.006253,0.002489,-0.000589,0.0,...,0.012392,0.006388,0.007633,0.005707,0.021359,0.009710,0.007511,0.009325,0.012151,0.014402
2020-08-28,0.009601,0.023723,NaN,0.002593,0.014265,0.008043,0.007635,0.002770,0.006303,0.0,...,0.010967,0.007819,0.007734,0.001892,0.021863,0.008656,0.008225,0.020035,0.011526,0.018752
2020-08-29,0.013946,0.017675,NaN,0.003135,0.013343,0.006672,0.004705,0.000000,0.015571,0.0,...,0.009714,0.011943,0.008990,0.010069,0.040000,0.010342,0.007885,0.014655,0.011070,-0.002104


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-27      0.002342  0.007921 -0.003247  0.003899  0.011364  0.001901   
2020-08-28      0.009346  0.007383  0.009772  0.003883  0.004086  0.005693   
2020-08-29      0.010031  0.018203  0.006452  0.011605  0.021363  0.009434   

Province_State                                          ...  Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park Platte   
2020-08-27      0.004944  0.004515 -0.002320  0.015106  ...  0.00000    0.0   
2020-08-28      0.001230  0.013483  0.003488  0.000000  ...  0.00000    0.0   
2020-08-29      0.015971  0.027051  0.004635  0.014881  ... -0.00625    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2020-08-27      0.000000      0.0    0.00000  0.000000  0.000000        inf   
2020-08-28      0.110236      0.0    0.00339  0.016787  0.010381   0.000000   
2020-08-29      0.042553      0.0    0.00000  0.002358  0.000000   0.377358   

Province_State                  
Admin2         Washakie Weston  
2020-08-27          0.0    0.0  
2020-08-28          0.0    0.0  
2020-08-29          0.0    0.0  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,0.015159,0.0,NaN,0.006740,0.009563,0.011570,0.002076,0.000448,0.0,NaN,...,0.015170,0.018534,0.004988,0.0,0.0,0.004771,0.005319,0.042105,0.010000,0.0
2020-08-28,0.014933,0.0,NaN,0.009941,0.023004,0.010097,0.003107,0.000000,0.0,NaN,...,0.016736,0.013910,0.009926,0.0,0.0,0.009102,0.007937,0.015152,0.001800,0.0
2020-08-29,0.021357,0.0,NaN,0.005826,0.021164,0.006950,0.002581,0.000000,0.0,NaN,...,0.014109,0.005583,0.000000,0.0,0.0,0.007059,0.000000,0.069652,0.005391,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb    Blount Bullock Butler   
2020-08-27         0.0  0.030303     0.0  0.0  0.000000     0.0    0.0   
2020-08-28         0.0  0.029412     0.0  0.0  0.285714     0.0    0.0   
2020-08-29         0.0  0.028571     0.0  0.2  0.000000     0.0    0.0   

Province_State                                ... Wyoming                  \
Admin2           Calhoun  Chambers  Cherokee  ...    Park Platte Sheridan   
2020-08-27      0.040000  0.000000  0.222222  ...     NaN    NaN      NaN   
2020-08-28      0.038462  0.026316  0.000000  ...     NaN    NaN      NaN   
2020-08-29      0.074074  0.000000  0.000000  ...     NaN    NaN      NaN   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-08-27          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-28          NaN        NaN   NaN   NaN        0.0      NaN    NaN  
2020-08-29          NaN        NaN   NaN   NaN        0.0      NaN    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.000733,0.018268,0.009291,0.010065,0.029330,0.000022,0.026947,0.003741,0.005151,0.011744,...,0.004057,0.004521,0.010409,0.020903,0.003647,0.024398,7.529182e-22,0.002371,0.015185,0.010263
2020-08-28,0.000511,0.015299,0.009144,0.016872,0.026259,0.000011,0.028879,0.003962,0.004521,0.010216,...,0.003979,0.003872,0.008579,0.021885,0.002789,0.026079,3.764591e-22,0.003772,0.015264,0.012760
2020-08-29,0.000295,0.012217,0.008927,0.008436,0.029317,0.000005,0.026212,0.003995,0.004668,0.012535,...,0.003652,0.006435,0.008551,0.023101,0.002358,0.022851,1.882295e-22,0.002658,0.012853,0.007789


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.001601,0.014160,0.006704,2.934383e-07,0.018132,2.313824e-41,0.030753,0.003657,0.028670,0.000140,...,0.000314,0.003292,0.023107,0.020773,0.029612,0.025981,0.0,0.004370,0.001325,0.057160
2020-08-28,0.001158,0.016478,0.006064,1.467191e-07,0.013828,1.156912e-41,0.029103,0.004722,0.028915,0.000070,...,0.000265,0.001646,0.018265,0.021560,0.014806,0.027175,0.0,0.003075,0.002436,0.044453
2020-08-29,0.000579,0.015619,0.005729,7.335956e-08,0.011631,5.784560e-42,0.019509,0.004087,0.023624,0.000035,...,0.000277,0.012451,0.017410,0.023075,0.040736,0.020484,0.0,0.001537,0.002985,0.024791


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-08-27,0.006466,0.028986,0.009096,0.004722,0.203380,0.000065,0.021801,0.007134,0.013072,0.008873,...,0.004558,0.009529,0.017765,0.034592,0.025077,0.040457,4.411630e-24,0.002924,0.019469,0.006811
2020-08-28,0.003457,0.028269,0.009174,0.007400,-0.013291,0.000032,0.021297,0.007298,0.013067,0.010642,...,0.018583,0.011891,0.017928,0.029376,0.032947,0.023646,2.205815e-24,0.005560,0.014578,0.006699
2020-08-29,0.001797,0.023986,0.009203,0.003700,-0.000323,0.000016,0.023239,0.007661,0.012035,0.011340,...,0.011459,0.010384,0.013941,0.031887,0.027031,0.023718,1.102907e-24,0.005490,0.014004,0.004638


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,0.013596,0.013502,NaN,0.002743,0.011028,0.007817,0.005632,0.002804,0.000616,1.455804e-50,...,0.011242,0.008030,0.007651,0.004734,0.022761,0.008667,0.006724,0.010611,0.010836,0.012586
2020-08-28,0.011599,0.018613,NaN,0.002668,0.012646,0.007930,0.006633,0.002787,0.003460,7.279019e-51,...,0.011104,0.007924,0.007692,0.003313,0.022312,0.008661,0.007475,0.015323,0.011181,0.015669
2020-08-29,0.012772,0.018144,NaN,0.002901,0.012995,0.007301,0.005669,0.001393,0.009515,3.639509e-51,...,0.010409,0.009934,0.008341,0.006691,0.031156,0.009502,0.007680,0.014989,0.011126,0.006783


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-08-27      0.001466  0.009253 -0.006634  0.003882  0.009901  0.000586   
2020-08-28      0.005406  0.008318  0.001569  0.003883  0.006994  0.003139   
2020-08-29      0.007718  0.013261  0.004010  0.007744  0.014178  0.006287   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2020-08-27      0.004092  0.008371 -0.002296  0.006125  ...  0.00382   
2020-08-28      0.002661  0.010927  0.000596  0.003062  ...  0.00191   
2020-08-29      0.009316  0.018989  0.002616  0.008972  ... -0.00217   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton    Uinta   
2020-08-27      0.000169  0.013394  0.002690   0.002399  0.004730  0.00534   
2020-08-28      0.000084  0.061815  0.001345   0.002894  0.010758  0.00786   
2020-08-29      0.000042  0.052184  0.000672   0.001447  0.006558  0.00393   

Province_State                                    
Admin2            Unassigned  Washakie    Weston  
2020-08-27     -1.000000e+00  0.000598 -0.002834  
2020-08-28     -1.164153e-10  0.000299 -0.001411  
2020-08-29      2.515723e-01  0.000150 -0.000704  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-08-27,0.010023,0.024408,NaN,0.009774,0.016941,0.011222,0.001760,0.000313,0.000697,NaN,...,0.015360,0.016737,0.008292,4.216044e-09,0.056589,0.006182,0.004210,0.031827,0.00814,0.000951
2020-08-28,0.012478,0.012204,NaN,0.009858,0.019973,0.010659,0.002434,0.000157,0.000348,NaN,...,0.016048,0.015323,0.009109,2.108022e-09,0.028295,0.007642,0.006073,0.023489,0.00497,0.000476
2020-08-29,0.016918,0.006102,NaN,0.007842,0.020568,0.008805,0.002507,0.000078,0.000174,NaN,...,0.015079,0.010453,0.004554,1.054011e-09,0.014147,0.007350,0.003037,0.046571,0.00518,0.000238


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                          \
Admin2               Autauga   Baldwin   Barbour      Bibb    Blount   
2020-08-27      3.692650e-07  0.023381  0.000327 -0.040885  0.043231   
2020-08-28      1.846325e-07  0.026397  0.000164 -0.020443  0.164473   
2020-08-29      9.231625e-08  0.027484  0.000082  0.089779  0.082236   

Province_State                                                           ...  \
Admin2           Bullock        Butler   Calhoun      Chambers Cherokee  ...   
2020-08-27      0.000089  2.725525e-08  0.035998  9.098084e-12   0.1116  ...   
2020-08-28      0.000044  1.362762e-08  0.037230  1.315789e-02   0.0558  ...   
2020-08-29      0.000022  6.813812e-09  0.055652  6.578947e-03   0.0279  ...   

Province_State Wyoming                                                  \
Admin2            Park Platte Sheridan Sublette Sweetwater Teton Uinta   
2020-08-27         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-28         NaN    NaN      NaN      NaN        NaN   NaN   NaN   
2020-08-29         NaN    NaN      NaN      NaN        NaN   NaN   NaN   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-08-27       0.000981      NaN    NaN  
2020-08-28       0.000491      NaN    NaN  
2020-08-29       0.000245      NaN    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200829,AK,6035,339660,NaN,43.0,NaN,NaN,NaN,7.0,...,345695,0,0,d14280fc719ed7f9365e0e046b0ac9ffd9a0b06b,0,0,0,0,0,NaN
1,20200829,AL,123889,851929,NaN,986.0,14267.0,NaN,1459.0,NaN,...,975818,45,0,668b17e7da32941aa4a95ef2775527e9e7d5d7a7,0,0,0,0,0,NaN
2,20200829,AR,60378,646592,NaN,407.0,4142.0,NaN,NaN,95.0,...,706970,16,0,19baf0f83cf44094d1194521bfebecbcdeaf6961,0,0,0,0,0,NaN
3,20200829,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e063eb477b842be4d1532f74fc5371dd23ec577d,0,0,0,0,0,NaN
4,20200829,AZ,201287,991089,NaN,812.0,21433.0,262.0,NaN,164.0,...,1192376,29,5,d6aca9f355470d3f2ed666030c62bc76e880d557,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200829,AK,6035.0,339660.0,NaN,43.0,NaN,NaN,NaN,7.0,...,345695,0,0,d14280fc719ed7f9365e0e046b0ac9ffd9a0b06b,0,0,0,0,0,NaN
1,20200829,AL,123889.0,851929.0,NaN,986.0,14267.0,NaN,1459.0,NaN,...,975818,45,0,668b17e7da32941aa4a95ef2775527e9e7d5d7a7,0,0,0,0,0,NaN
2,20200829,AR,60378.0,646592.0,NaN,407.0,4142.0,NaN,NaN,95.0,...,706970,16,0,19baf0f83cf44094d1194521bfebecbcdeaf6961,0,0,0,0,0,NaN
3,20200829,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e063eb477b842be4d1532f74fc5371dd23ec577d,0,0,0,0,0,NaN
4,20200829,AZ,201287.0,991089.0,NaN,812.0,21433.0,262.0,NaN,164.0,...,1192376,29,5,d6aca9f355470d3f2ed666030c62bc76e880d557,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-29,AK,6035,339660,NaN,43.0,NaN,NaN,NaN,7.0,NaN,...,345695,0,0,d14280fc719ed7f9365e0e046b0ac9ffd9a0b06b,0,0,0,0,0,NaN
2020-08-29,AL,123889,851929,NaN,986.0,14267.0,NaN,1459.0,NaN,801.0,...,975818,45,0,668b17e7da32941aa4a95ef2775527e9e7d5d7a7,0,0,0,0,0,NaN
2020-08-29,AR,60378,646592,NaN,407.0,4142.0,NaN,NaN,95.0,538.0,...,706970,16,0,19baf0f83cf44094d1194521bfebecbcdeaf6961,0,0,0,0,0,NaN
2020-08-29,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e063eb477b842be4d1532f74fc5371dd23ec577d,0,0,0,0,0,NaN
2020-08-29,AZ,201287,991089,NaN,812.0,21433.0,262.0,NaN,164.0,NaN,...,1192376,29,5,d6aca9f355470d3f2ed666030c62bc76e880d557,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-08-29,AK,6035.0,339660.0,NaN,43.0,NaN,NaN,NaN,7.0,NaN,...,345695,0,0,d14280fc719ed7f9365e0e046b0ac9ffd9a0b06b,0,0,0,0,0,NaN
2020-08-29,AL,123889.0,851929.0,NaN,986.0,14267.0,NaN,1459.0,NaN,801.0,...,975818,45,0,668b17e7da32941aa4a95ef2775527e9e7d5d7a7,0,0,0,0,0,NaN
2020-08-29,AR,60378.0,646592.0,NaN,407.0,4142.0,NaN,NaN,95.0,538.0,...,706970,16,0,19baf0f83cf44094d1194521bfebecbcdeaf6961,0,0,0,0,0,NaN
2020-08-29,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,e063eb477b842be4d1532f74fc5371dd23ec577d,0,0,0,0,0,NaN
2020-08-29,AZ,201287.0,991089.0,NaN,812.0,21433.0,262.0,NaN,164.0,NaN,...,1192376,29,5,d6aca9f355470d3f2ed666030c62bc76e880d557,0,0,0,0,0,NaN


# TODO - CONTINUE FROM HERE